In [6]:
min_suport = 0.4

# mlxtend

In [7]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [8]:
path = 'dataset2.csv'
dataframe = pd.read_csv(path)
dataframe = dataframe.drop(columns=['videoCategoryId', 'definition'])
dataframe = dataframe.fillna(dataframe.mode().iloc[0])
t = {} 
for k, v in dataframe.values:
    if k in t.keys():
        if v not in t[k]:
            t[k] += [v]
    else:
        t[k] = [v] 
te = TransactionEncoder()
t = [[k] + v for k, v in t.items()]
te_ary = te.fit(t).transform(t)
dataframe = pd.DataFrame(te_ary, columns=te.columns_)

In [9]:
result = apriori(dataframe, min_support=min_suport, use_colnames=True)
result['support'] *= len(dataframe)
result = result.sort_values(by=['support'], ascending=False)
result.reset_index(drop=True, inplace=True)
result

,support,itemsets
0,31.0,(People & Blogs)
1,24.0,(Science & Technology)
2,17.0,"(Science & Technology, People & Blogs)"


In [10]:
association_rules(result, metric="support", min_threshold=0) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Science & Technology),(People & Blogs),24.0,31.0,17.0,0.708333,0.022849,-727.0,-102.857143
1,(People & Blogs),(Science & Technology),31.0,24.0,17.0,0.548387,0.022849,-727.0,-50.928571


# Ours

In [11]:
from DataFrame import DF
import itertools

In [12]:
def get_data(df, clean=False):
    df.missing_values_imputation()
    df.encode_str()
    if clean:
        for att in list(df.data.keys()):
            df.outliers_median_imputation(att)
        for att in list(df.data.keys()):
            df.normalize_minmax(att)
    return [[df.encode_dict[att][df.data[att][i]] for att in list(df.data.keys())] for i in range(len(df.data.get('Watcher')))]

df = DF(remove=['definition', 'videoCategoryId'])
transactions = {}
for k, v in get_data(df):
    if k in transactions.keys():
        if v not in transactions[k]:
            transactions[k] += tuple([v])
    else:
        transactions[k] = tuple([v]) 
transactions = list(transactions.values())

In [13]:
def Apriori(transactions, min_support = 0.2):
    out = {}
    c = {}
    min_support = min_support * len(transactions)

    for transaction in transactions:
        for item in transaction: 
            c[(item,)] = c[(item,)] + 1 if (item,) in c.keys() else 1
    l = {k: v for k, v in c.items() if v >= min_support}
    out.update(l)
    
    while c:
        c = {} 
        for item_set1 in list(l.keys()):
            for item_set2 in list(l.keys()):
                if item_set1 != item_set2 and item_set1 != item_set2[::-1]: 
                    c[tuple(set(item_set1) | set(item_set2))] = 0   
        for transaction in transactions:
            for item_set in c.keys():
                if (set(item_set).issubset(transaction)):
                    c[item_set] = c[item_set] + 1
        l = {k: v for k, v in c.items() if v >= min_support}
        out.update(l) 
    out = dict(sorted(out.items(), key=lambda item: item[1], reverse=True))
    return l, c, out

In [22]:
l, c, out = Apriori(transactions, 0.1)  
output = pd.DataFrame([[k, v] for k, v in zip(out.keys(), out.values())]) 
output.columns = ['itemsets', 'support']
output 

,itemsets,support
0,"(People & Blogs,)",31
1,"(Science & Technology,)",24
2,"(Science & Technology, People & Blogs)",17
3,"(Nonprofits & Activism,)",9
4,"(Science & Technology, Nonprofits & Activism)",9
5,"(Education,)",8
6,"(Nonprofits & Activism, People & Blogs)",8
7,"(Science & Technology, Nonprofits & Activism, ...",8
8,"(Entertainment,)",7
9,"(People & Blogs, Education)",6


In [23]:
def possibilities(L):
    if len(L) == 1:
        return
    perm = tuple(itertools.permutations(L)) 
    out = {}
    for p in perm:
        for i in range(1, len(p)):
            out[p[:i]] = p[i:]
    return out

In [49]:
def get_rules(transactions, out, min_support=0.2, min_conf=0):
    rules = []
    for itemset, support in out.items():
        if len(itemset) > 1:
            for antecedents, consequents in possibilities(itemset).items():
                try:
                    if consequents not in out.keys():
                        consequents = consequents[::-1]
                    support = out[tuple(set(antecedents) | set(consequents))]
                    confidence = out[tuple(set(antecedents) | set(consequents))] / out[antecedents]
                    lift = out[tuple(set(antecedents) | set(consequents))] / (out[antecedents] * out[consequents])
                    rules.append([antecedents, consequents, support, confidence, lift])
                except:
                    pass    
    if min_support:
        min_support = min_support * len(transactions)
        rules = [rule for rule in rules if rule[2] >= min_support]
    if min_conf:
        rules = [rule for rule in rules if rule[3] >= min_conf]
    return rules    

rules_ = get_rules(transactions, out)
rules = pd.DataFrame(rules_)  
rules.columns = ['antecedents', 'consequents', 'support', 'confidence', 'lift'] 
rules = rules.sort_values(by=['support'], ascending=False)
rules.reset_index(drop=True, inplace=True)
rules

,antecedents,consequents,support,confidence,lift
0,"(Science & Technology,)","(People & Blogs,)",17,0.708333,0.022849
1,"(People & Blogs,)","(Science & Technology,)",17,0.548387,0.022849
2,"(Science & Technology,)","(Nonprofits & Activism,)",9,0.375000,0.041667
3,"(Nonprofits & Activism,)","(Science & Technology,)",9,1.000000,0.041667
4,"(Nonprofits & Activism,)","(People & Blogs,)",8,0.888889,0.028674
5,"(People & Blogs,)","(Nonprofits & Activism,)",8,0.258065,0.028674
6,"(Science & Technology,)","(Nonprofits & Activism, People & Blogs)",8,0.333333,0.041667
7,"(Science & Technology, Nonprofits & Activism)","(People & Blogs,)",8,0.888889,0.028674
8,"(Science & Technology, People & Blogs)","(Nonprofits & Activism,)",8,0.470588,0.052288
9,"(Nonprofits & Activism,)","(Science & Technology, People & Blogs)",8,0.888889,0.052288


In [64]:
transactions[5]

('News & Politics',
 'Science & Technology',
 'Entertainment',
 'Education',
 'Gaming')

In [63]:
def get_pred(watcher=5, transactions=transactions, rules=rules_):
    change = True
    interests = transactions[watcher]
    while(change):
        change = False
        for rule in rules:
            if set(rule[0]).issubset(set(interests)):
                if not set(rule[1]).issubset(set(interests)):
                    interests = set(interests) | set(rule[1])
                    change = True
    return [i for i in interests if i not in transactions[watcher]]
get_pred()

['Nonprofits & Activism', 'People & Blogs']